# Experiment: Reconstruction from Speleo-Pseudoproxies with different PSMs

Inspiration: https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2016MS000677

- We use all Speleo-locations (100 smething) (first without karst filter)
- no height correction (this is not useful here for pseudoproxies
- real-world snr (0.5, 5 with karst-filter) (beware that the SNR is having way more impact when using karst filter)
- the reconstruction psm uses different types of configurations (varying/mean temperature, infiltration/prec weighting/None, karst filter/on-off)

Wrapper for all models, mme, mme with leave one out.


**Visualization:**
 - Box-Plot of all CEs, R,s (One plot with six configurations for every ) (instead of a swarm plot)
 - Reconstruction of global mean temperature from different configs (who does it change with the Karst-Filter)
 - Maps for different metrics (Then it will get messy)
 
**Number of experiment configs in total**
- pseudoproxy creation: Karst off (SNR 0.5), Karst on (SNR 5, or even 10?) The noise is applied after the karst-filter, so the errors get hugehere
- 2 karst on/off x 3x inf/prec/None weighting x 2 varying mean temperature = 12

12 * 5 + for each model try a multi-model (leave one out reconstruction)

 
**HINT** Give the output folder similar names, such that for making the plots the paths can be adapted easily with control H. (search and replace)

In [1]:
import sys
sys.path.append('/home/mchoblet/paleoda')

import xarray as xr
import wrapper
import dataloader
import tqdm
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import cartopy.feature as cfeature  
from matplotlib.offsetbox import AnchoredText    
import utils

import copy

%load_ext autoreload
%autoreload 2

In [2]:
###CONFIG DICTIONARY

#lists are sometimes necessary, especially if we use multiple proxy-DBs (e.g. SISAL and iso2k)
#and multiple timescales

cfg = {
#BASIC PARAMS
'vp':{
'd18O':'/home/mchoblet/paleoda/data/model_data/ECHAM5_d18O_850_1849.nc',
'prec':'/home/mchoblet/paleoda/data/model_data/ECHAM5_prec_850_1849.nc',
'tsurf': '/home/mchoblet/paleoda/data/model_data/ECHAM5_tsurf_850_1849.nc',
'evap': '/home/mchoblet/paleoda/data/model_data/ECHAM5_evap_850_1849.nc',
'slp': None,
},
'oro': None,
'reconstruct':['tsurf','d18O','prec'], 
'output_folder':'PPE_speleopsm', #in cwd
'output_file':'dummy',
 'multi_model_prior': None, #None or a list where the first entry is a dictionary of dictionaries for the file paths and secord a dictionary with the orography files
'avg':None, 
'seed': 42, 
'check_nan':True, #mainly needed for iHadCM3, because of its missing years nans are introduced with 'resampled'
###PROXYS
'obsdata':['/home/mchoblet/paleoda/data/proxy_dbs/sisal_db.nc'],
'obs_var':[['d18O','d18O_precision']], #name of variables and uncertainties in proxy_db files
'proxy_error': [0.1], #irrelevant for PPE
'time_scales': ['median'], 
'proxy_time':['0851','1849'], 
'resample': True,
'resample_mode': 'nearest',
'mask': 3, 
###MULTI-TIMESCALE
'timescales':[1], #list of timescales (have to be true divisors of longest timescale (block size)) Timescale 1 always required! 
'reuse': True, #e.g if a yearly resolved proxy is also to be used in the 5,10 year mean reconstruction
###PSM
'psm':[None], 
'interpol': 'dw',#None, #None/'dw'
'var_psm': ['d18O'] ,
'speleo':{
    'weighting':'inf', #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation':True, #requires tsurf (in 'vp')
    'fractionation_temp': 'mean', #'regular'
    'filter': False, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
},
'icecore':{
    'weighting':'prec', #None(pure d18O) / prec (d180 prec weighted)
    'height': False, #height correction: True requires orography of model and 'elev' coordinate for proxy_db. Requires that proxy_db has a coordinate 'lapse rate' (e.g. i attributed -0.15 to non polar porxies and -0.28 to polar ones (per 100m)
    'filter': False, #If True the PSM-function from PRYSM which takes into account diffusion is applied. 
},
#currently this only works for one Database files, probably at some point that needs to be more flexible
'linear':{
    'variable': 'tsurf' , #variable to regress against
    'type': 'anom' , #absolute value or anomaly
    'anom_time': None , #None for full simulation, or time-list 
    'uniform': None, # if you just want one linear formula to rule them all [slope, intercept]
    #names of the regression parameters in the proxy-db
    'slope': 'slope' ,
    'intercept': 'intercept',
    'error': 'error', #make sure that the error in the database info is really the regression error *variance*, not standard error.
},
###PSEUDOPROXY-EXPERIMENTS (variable comes from var_psm)
'ppe': {
    'use': True,
    'SNR': [0.5], #signal to noise variance ratio
    'noise_type': 'w', #'w'/'r'
    'red_noise_a': 0.32, #autocorrelation for red noise
    'multi_timescale': [[1],[1]], 
    'source': 'external', #internal/external
    'other_model': 
    {'vp':{'tsurf':'/home/ldap-server/draco/cleaned_data/final/CESM_tsurf_850_1850.nc',
    'prec':'/home/ldap-server/draco/cleaned_data/final/CESM_prec_850_1850.nc',
    'd18O':'/home/ldap-server/draco/cleaned_data/final/CESM_d18O_850_1850.nc',
    'evap': '/home/ldap-server/draco/cleaned_data/final/CESM_evap_850_1850.nc'}},
    'oro': '/home/ldap-server/draco/orographies/final/icesm_oro.nc',
    'metrics_ppe': ['Corr','p_value','p_bootstrap','eff_p_value','CE','RMSE','RMSPE','MAE','MAPE'],
    'psm' : {'speleo':{   'weighting':'inf', #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation':True, #requires tsurf (in 'vp')
    'fractionation_temp': 'regular', #'regular'
    'filter': False, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
}, 'icecore': None},
},
    # cfg['ppe']['psm']= {'speleo':None, 'icecore': None} # if the config from above for the psm (which gives the obs-from-model) shall be different for the pseudoproxies.
#else just leave it as {'speleo':None, 'icecore': None} 
    
###KALMAN-FILTER
'time': ['0851','1849'], #covered by all models
'nens':100, #ensemble members
'reps':5, #Monte Carlo
'full_fields': True, 
'proxy_frac': None, #choose 100 random proxies
'extra_assi': 
    {
        'globalmean':['tsurf','prec','d18O'], #None or list of variables
        'lat_mean': None, #None or list of variables
        'regionalmean': None, #None or list of variables
        'region':None, #None or list of list [[latS,latN],[latW,latE]] #specify the regions in 0-360 longitude system
        'prec_weighted': False, #if you want to get the precipitation weighted d18O
    },
'error_scale':1, 
'anomaly': 'BG+HX', #/ 'HX'
'anomaly_time': None,
'cov_loc': None,
#evaluation metrics (which do not rely on a true state)
'metrics': [None],#['rank_histogram','rank_histogram_posterior','CD'], #['None']
}

In [3]:
#regular resolution of model data

multi_prior=[{
'iCESM':{'d18O':'/home/mchoblet/paleoda/data/model_data/CESM_d18O_850_1850.nc',
'evap':'/home/mchoblet/paleoda/data/model_data/CESM_evap_850_1850.nc',
'prec':'/home/mchoblet/paleoda/data/model_data/CESM_prec_850_1850.nc',
'tsurf':'/home/mchoblet/paleoda/data/model_data/CESM_tsurf_850_1850.nc',
       },
'isoGSM':{'d18O':'/home/mchoblet/paleoda/data/model_data/CCSM_d18O_851_1850.nc',
'evap':'/home/mchoblet/paleoda/data/model_data/CCSM_evap_851_1850.nc',
'prec':'/home/mchoblet/paleoda/data/model_data/CCSM_prec_851_1850.nc',
'tsurf':'/home/mchoblet/paleoda/data/model_data/CCSM_tsurf_851_1850.nc',
       },    
'Echam':{'d18O':'/home/mchoblet/paleoda/data/model_data/ECHAM5_d18O_850_1849.nc',
'evap':'/home/mchoblet/paleoda/data/model_data/ECHAM5_evap_850_1849.nc',
'prec':'/home/mchoblet/paleoda/data/model_data/ECHAM5_prec_850_1849.nc',
'tsurf':'/home/mchoblet/paleoda/data/model_data/ECHAM5_tsurf_850_1849.nc',
       },
'GISS':{'d18O':'/home/mchoblet/paleoda/data/model_data/GISS_d18O_850_1849.nc',
'evap':'/home/mchoblet/paleoda/data/model_data/GISS_evap_850_1849.nc',
'prec':'/home/mchoblet/paleoda/data/model_data/GISS_prec_850_1849.nc',
'tsurf':'/home/mchoblet/paleoda/data/model_data/GISS_tsurf_850_1849.nc',
'slp': '/home/mchoblet/paleoda/data/model_data/GISS_slp_850_1849.nc',
       },
'iHadCM3':{'d18O':'/home/mchoblet/paleoda/data/model_data/iHADCM3_d18O_801_1952.nc',
'evap':'/home/mchoblet/paleoda/data/model_data/iHADCM3_evap_801_1952.nc',
'prec':'/home/mchoblet/paleoda/data/model_data/iHADCM3_prec_801_1952.nc',
'tsurf':'/home/mchoblet/paleoda/data/model_data/iHADCM3_tsurf_801_1952.nc',
'slp':'/home/mchoblet/paleoda/data/model_data/iHADCM3_slp_801_1952.nc',
       }
},
#I checked once again that nothing with the longitudes is wrong (all go from 0 to 360)
{
    'isoGSM':'/home/mchoblet/paleoda/data/orographies/ccsm_oro.nc',
'Echam': '/home/mchoblet/paleoda/data/orographies/echam_oro.nc',
'GISS':'/home/mchoblet/paleoda/data/orographies/giss_oro.nc',
'iHadCM3':'/home/mchoblet/paleoda/data/orographies/hadcm3_oro.nc',
'iCESM':'/home/mchoblet/paleoda/data/orographies/icesm_oro.nc',
}]


multi_prior_ds=[{
'iCESM':{'d18O':'/home/mchoblet/paleoda/data/model_data/downsampled/CESM_d18O_850_1850.nc',
'evap':'/home/mchoblet/paleoda/data/model_data/downsampled/CESM_evap_850_1850.nc',
'prec':'/home/mchoblet/paleoda/data/model_data/downsampled/CESM_prec_850_1850.nc',
'tsurf':'/home/mchoblet/paleoda/data/model_data/downsampled/CESM_tsurf_850_1850.nc',
       },
'isoGSM':{'d18O':'/home/mchoblet/paleoda/data/model_data/downsampled/CCSM_d18O_851_1850.nc',
'evap':'/home/mchoblet/paleoda/data/model_data/downsampled/CCSM_evap_851_1850.nc',
'prec':'/home/mchoblet/paleoda/data/model_data/downsampled/CCSM_prec_851_1850.nc',
'tsurf':'/home/mchoblet/paleoda/data/model_data/downsampled/CCSM_tsurf_851_1850.nc',
       },    
'Echam':{'d18O':'/home/mchoblet/paleoda/data/model_data/downsampled/ECHAM5_d18O_850_1849.nc',
'evap':'/home/mchoblet/paleoda/data/model_data/downsampled/ECHAM5_evap_850_1849.nc',
'prec':'/home/mchoblet/paleoda/data/model_data/downsampled/ECHAM5_prec_850_1849.nc',
'tsurf':'/home/mchoblet/paleoda/data/model_data/downsampled/ECHAM5_tsurf_850_1849.nc',
       },
'GISS':{'d18O':'/home/mchoblet/paleoda/data/model_data/downsampled/GISS_d18O_850_1849.nc',
'evap':'/home/mchoblet/paleoda/data/model_data/downsampled/GISS_evap_850_1849.nc',
'prec':'/home/mchoblet/paleoda/data/model_data/downsampled/GISS_prec_850_1849.nc',
'tsurf':'/home/mchoblet/paleoda/data/model_data/downsampled/GISS_tsurf_850_1849.nc',
'slp': '/home/mchoblet/paleoda/data/model_data/downsampled/GISS_slp_850_1849.nc',
       },
'iHadCM3':{'d18O':'/home/mchoblet/paleoda/data/model_data/downsampled/iHADCM3_d18O_801_1952.nc',
'evap':'/home/mchoblet/paleoda/data/model_data/downsampled/iHADCM3_evap_801_1952.nc',
'prec':'/home/mchoblet/paleoda/data/model_data/downsampled/iHADCM3_prec_801_1952.nc',
'tsurf':'/home/mchoblet/paleoda/data/model_data/downsampled/iHADCM3_tsurf_801_1952.nc',
'slp':'/home/mchoblet/paleoda/data/model_data/downsampled/iHADCM3_slp_801_1952.nc',
       }
},
#I checked once again that nothing with the longitudes is wrong (all go from 0 to 360)
{
    'isoGSM':'/home/mchoblet/paleoda/data/orographies/ccsm_oro.nc',
'Echam': '/home/mchoblet/paleoda/data/orographies/echam_oro.nc',
'GISS':'/home/mchoblet/paleoda/data/orographies/giss_oro.nc',
'iHadCM3':'/home/mchoblet/paleoda/data/orographies/hadcm3_oro.nc',
'iCESM':'/home/mchoblet/paleoda/data/orographies/icesm_oro.nc',
}]



In [4]:
cfg['var_psm']

['d18O']

In [13]:
def speleo_pseudoproxy_nokarst(paths,cfg):
    """
    Paths is the multi-prior variable (where the second one is orography just in case). Hence treat first element of paths, paths[0] as the dict with paths.
    
    """
    cfg_exp=cfg.copy()
    cfg_exp['output_folder']='PPE_speleopsm_nokarst'
    cfg_exp['reps']=1
    cfg_exp['psm']=['speleo']
    cfg_exp['ppe']['psm']['speleo']={
    'weighting':'inf', #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation':True, #requires tsurf (in 'vp')
    'fractionation_temp': 'regular', #'regular'
    'filter': False, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
    }
    
    #loop over proxy origin
    for i1,p1 in tqdm.tqdm(paths[0].items()):
        
        cfg_exp['vp']=p1
        cfg_exp['multi_model_prior']=None
        cfg_exp['ppe']['other_model']['vp']=p1
        
        #1. no-psm at all

        cfg_exp['speleo']={ 'weighting':None, #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
            'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
            'fractionation': False, #requires tsurf (in 'vp')
            'fractionation_temp': 'regular', #'regular'
            'filter': False, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
            't_time': 2.5, #if filter equal true, use this as a transit time
        }

        cfg_exp['output_file']=str(i1)+'_PP_'+'no_psm'
        wrapper.paleoda_wrapper(cfg_exp)

        #2. infiltration weighting
        cfg_exp['speleo']={
    'weighting':'inf', #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation': False, #requires tsurf (in 'vp')
    'fractionation_temp': 'regular', #'regular'
    'filter': False, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'inf'
        wrapper.paleoda_wrapper(cfg_exp)

        #3. infiltration weighting + fractionation (mean)
        cfg_exp['speleo']={
    'weighting':'inf', #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation': True, #requires tsurf (in 'vp')
    'fractionation_temp': 'mean', #'regular'
    'filter': False, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'inf_frac_m'
        wrapper.paleoda_wrapper(cfg_exp)


        #4. inf + fractionation regular
        cfg_exp['speleo']={
    'weighting':'inf', #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation': True, #requires tsurf (in 'vp')
    'fractionation_temp': 'regular', #'regular'
    'filter': False, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
}

        cfg_exp['output_file']=str(i1)+'_PP_'+'inf_frac_reg'
        wrapper.paleoda_wrapper(cfg_exp)
        
        #the same using a multi- model prior (leave original model out)
        copied_paths=copy.deepcopy(multi_prior_ds[0])
        #delete current proxy origin
        del copied_paths[i1]
        #put back into list form
        copied_paths=[copied_paths]
        cfg_exp['multi_model_prior']=copied_paths

        #1. no-psm at all
        
        cfg_exp['speleo']={
    'weighting':None, #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation': False, #requires tsurf (in 'vp')
    'fractionation_temp': 'regular', #'regular'
    'filter': False, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'no_psm_mme'
        wrapper.paleoda_wrapper(cfg_exp)
        
        #2. infiltration weighting
        cfg_exp['speleo']={
    'weighting':'inf', #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation': False, #requires tsurf (in 'vp')
    'fractionation_temp': 'regular', #'regular'
    'filter': False, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'inf_mme'
        wrapper.paleoda_wrapper(cfg_exp)
        
        #3. infiltration weighting + fractionation (mean)
        cfg_exp['speleo']={
    'weighting':'inf', #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation': True, #requires tsurf (in 'vp')
    'fractionation_temp': 'mean', #'regular'
    'filter': False, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'inf_frac_m_mme'
        wrapper.paleoda_wrapper(cfg_exp)
        
        
        #4. inf + fractionation regular
        cfg_exp['speleo']={
    'weighting':'inf', #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation': True, #requires tsurf (in 'vp')
    'fractionation_temp': 'regular', #'regular'
    'filter': False, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
    }

        cfg_exp['output_file']=str(i1)+'_PP_'+'inf_frac_reg_mme'
        wrapper.paleoda_wrapper(cfg_exp)
        

In [14]:
speleo_pseudoproxy_nokarst(multi_prior_ds,cfg)

  0%|          | 0/5 [00:00<?, ?it/s]

>>>>>LOADING PROXY DATA
Load d18O  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_d18O_850_1850.nc
computing yearly average
Checking prior for nans
Load evap  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_evap_850_1850.nc
computing yearly average
Checking prior for nans
Load prec  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_prec_850_1850.nc
computing yearly average
Checking prior for nans
Load tsurf  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_tsurf_850_1850.nc
computing yearly average
Checking prior for nans
USING SPELEO PSM
COMPUTE ANOMALIES IN MODE: BG+HX
GENERATING PSEUDOPROXIES FROM MODEL OTHER THAN PRIOR
Changed Pseudoproxy-PSM-Config for  speleo
Load d18O  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_d18O_850_1850.nc
computing yearly average
Checking prior for nans
Load evap  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_evap_850_1850.nc
computing yearly average
Checking prior for


100%|██████████| 1/1 [00:13<00:00, 13.70s/it]


Finished multitimescale DA
Save variables
Compute pseudoproxy metrics for  tsurf
Compute pseudoproxy metrics for  d18O
Compute pseudoproxy metrics for  prec
Output written to iCESM_PP_no_psm.nc
>>>>>LOADING PROXY DATA
Load d18O  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_d18O_850_1850.nc
computing yearly average
Checking prior for nans
Load evap  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_evap_850_1850.nc
computing yearly average
Checking prior for nans
Load prec  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_prec_850_1850.nc
computing yearly average
Checking prior for nans
Load tsurf  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_tsurf_850_1850.nc
computing yearly average
Checking prior for nans
USING SPELEO PSM
>>>>>>>>>>>>>GETTING MONTHLY d18O Data
Checking prior for nans
COMPUTE ANOMALIES IN MODE: BG+HX
GENERATING PSEUDOPROXIES FROM MODEL OTHER THAN PRIOR
Changed Pseudoproxy-PSM-Config for  speleo
Load d18O  from


100%|██████████| 1/1 [00:13<00:00, 13.65s/it]


...

Finished multitimescale DA
Save variables
Compute pseudoproxy metrics for  tsurf
Compute pseudoproxy metrics for  d18O
Compute pseudoproxy metrics for  prec
Output written to iHadCM3_PP_inf_frac_m_mme.nc
>>>>>LOADING PROXY DATA
Will compute a multi-model-prior!
Load d18O  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_d18O_850_1850.nc
computing yearly average
Load evap  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_evap_850_1850.nc
computing yearly average
Load prec  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_prec_850_1850.nc
computing yearly average
Load tsurf  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_tsurf_850_1850.nc
computing yearly average
USING SPELEO PSM
>>>>>>>>>>>>>GETTING MONTHLY d18O Data
>>>>>>>>>>>>>APPLYING FRACTIONATION
use time-varying temperature
COMPUTE ANOMALIES IN MODE: BG+HX
Load d18O  from  /home/mchoblet/paleoda/data/model_data/downsampled/CCSM_d18O_851_1850.nc
computing yearly average
Load ev


100%|██████████| 1/1 [01:25<00:00, 85.10s/it]


Finished multitimescale DA
Save variables
Compute pseudoproxy metrics for  tsurf
Compute pseudoproxy metrics for  d18O
Compute pseudoproxy metrics for  prec


100%|██████████| 5/5 [1:21:13<00:00, 974.67s/it]

Output written to iHadCM3_PP_inf_frac_reg_mme.nc


In [75]:
cfg['extra_assi']

{'globalmean': ['tsurf', 'prec', 'd18O'],
 'lat_mean': None,
 'regionalmean': None,
 'region': None,
 'prec_weighted': False}

In [23]:
def speleo_pseudoproxy_karst(paths,cfg):
    """
    Paths is the multi-prior variable (where the second one is orography just in case). Hence treat first element of paths, paths[0] as the dict with paths.
    
    """
    cfg_exp=cfg.copy()
    cfg_exp['output_folder']='PPE_speleopsm_karst'
    cfg_exp['psm']=['speleo']
    cfg_exp['reps']=1
    cfg_exp['ppe']['psm']['speleo']={
    'weighting':'inf', #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation':True, #requires tsurf (in 'vp')
    'fractionation_temp': 'regular', #'regular'
    'filter': True, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
    }
    
    #here choose extremely high snr
    cfg_exp['ppe']['SNR']=[10]
    
    
    #loop over proxy origin
    for i1,p1 in tqdm.tqdm(paths[0].items()):
        cfg_exp['multi_model_prior']=None
        cfg_exp['ppe']['other_model']['vp']=p1
        """
        #1. no-psm at all
        
        cfg_exp['speleo']={
    'weighting':None, #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation': False, #requires tsurf (in 'vp')
    'fractionation_temp': 'regular', #'regular'
    'filter': True, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'no_psm'
        wrapper.paleoda_wrapper(cfg_exp)
        
        #2. infiltration weighting
        cfg_exp['speleo']={
    'weighting':'inf', #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation': False, #requires tsurf (in 'vp')
    'fractionation_temp': 'regular', #'regular'
    'filter': True, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'inf'
        wrapper.paleoda_wrapper(cfg_exp)
        
        #3. infiltration weighting + fractionation (mean)
        cfg_exp['speleo']={
    'weighting':'inf', #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation': True, #requires tsurf (in 'vp')
    'fractionation_temp': 'mean', #'regular'
    'filter': True, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'inf_frac_m'
        wrapper.paleoda_wrapper(cfg_exp)
        
        """
        #4. inf + fractionation regular
        cfg_exp['speleo']={
    'weighting':'inf', #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation': True, #requires tsurf (in 'vp')
    'fractionation_temp': 'regular', #'regular'
    'filter': True, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
}
        
        cfg_exp['output_file']=str(i1)+'_PP_'+'inf_frac_reg'
        wrapper.paleoda_wrapper(cfg_exp)
        
        
        #the same using a multi- model prior (leave original model out)
        copied_paths=copy.deepcopy(multi_prior_ds[0])
        #delete current proxy origin
        del copied_paths[i1]
        #put back into list form
        copied_paths=[copied_paths]
        cfg_exp['multi_model_prior']=copied_paths

                #1. no-psm at all
        """
        cfg_exp['speleo']={
    'weighting':None, #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation': False, #requires tsurf (in 'vp')
    'fractionation_temp': 'regular', #'regular'
    'filter': True, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'no_psm_mme'
        wrapper.paleoda_wrapper(cfg_exp)
        
        #2. infiltration weighting
        cfg_exp['speleo']={
    'weighting':'inf', #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation': False, #requires tsurf (in 'vp')
    'fractionation_temp': 'regular', #'regular'
    'filter': True, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'inf_mme'
        wrapper.paleoda_wrapper(cfg_exp)
        """
        #3. infiltration weighting + fractionation (mean)
        cfg_exp['speleo']={
    'weighting':'inf', #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation': True, #requires tsurf (in 'vp')
    'fractionation_temp': 'mean', #'regular'
    'filter': True, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'inf_frac_m_mme'
        wrapper.paleoda_wrapper(cfg_exp) 
        
        #4. inf + fractionation regular
        cfg_exp['speleo']={
    'weighting':'inf', #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation': True, #requires tsurf (in 'vp')
    'fractionation_temp': 'regular', #'regular'
    'filter': True, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
}
        
        cfg_exp['output_file']=str(i1)+'_PP_'+'inf_frac_reg_mme'
        wrapper.paleoda_wrapper(cfg_exp)
        

In [ ]:
speleo_pseudoproxy_karst(multi_prior_ds,cfg)

  0%|          | 0/5 [00:00<?, ?it/s]

>>>>>LOADING PROXY DATA
Load d18O  from  /home/mchoblet/paleoda/data/model_data/ECHAM5_d18O_850_1849.nc
computing yearly average
Checking prior for nans
Load prec  from  /home/mchoblet/paleoda/data/model_data/ECHAM5_prec_850_1849.nc
computing yearly average
Checking prior for nans
Load tsurf  from  /home/mchoblet/paleoda/data/model_data/ECHAM5_tsurf_850_1849.nc
computing yearly average
Checking prior for nans
Load evap  from  /home/mchoblet/paleoda/data/model_data/ECHAM5_evap_850_1849.nc
computing yearly average
Checking prior for nans
USING SPELEO PSM
>>>>>>>>>>>>>APPLYING KARST FILTER
COMPUTE ANOMALIES IN MODE: BG+HX
GENERATING PSEUDOPROXIES FROM MODEL OTHER THAN PRIOR
Changed Pseudoproxy-PSM-Config for  speleo
Load d18O  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_d18O_850_1850.nc
computing yearly average
Checking prior for nans
Load evap  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_evap_850_1850.nc
computing yearly average
Checking prior for nans


100%|██████████| 1/1 [00:12<00:00, 12.48s/it]


Finished multitimescale DA
Save variables
Compute pseudoproxy metrics for  tsurf
Compute pseudoproxy metrics for  d18O
Compute pseudoproxy metrics for  prec
Output written to iCESM_PP_no_psm.nc
>>>>>LOADING PROXY DATA
Load d18O  from  /home/mchoblet/paleoda/data/model_data/ECHAM5_d18O_850_1849.nc
computing yearly average
Checking prior for nans
Load prec  from  /home/mchoblet/paleoda/data/model_data/ECHAM5_prec_850_1849.nc
computing yearly average
Checking prior for nans
Load tsurf  from  /home/mchoblet/paleoda/data/model_data/ECHAM5_tsurf_850_1849.nc
computing yearly average
Checking prior for nans
Load evap  from  /home/mchoblet/paleoda/data/model_data/ECHAM5_evap_850_1849.nc
computing yearly average
Checking prior for nans
USING SPELEO PSM
>>>>>>>>>>>>>GETTING MONTHLY d18O Data
Checking prior for nans
>>>>>>>>>>>>>APPLYING KARST FILTER
COMPUTE ANOMALIES IN MODE: BG+HX
GENERATING PSEUDOPROXIES FROM MODEL OTHER THAN PRIOR
Changed Pseudoproxy-PSM-Config for  speleo
Load d18O  from  /ho


100%|██████████| 1/1 [00:12<00:00, 12.45s/it]


...

Finished multitimescale DA
Save variables
Compute pseudoproxy metrics for  tsurf
Compute pseudoproxy metrics for  d18O
Compute pseudoproxy metrics for  prec
Output written to isoGSM_PP_no_psm_mme.nc
>>>>>LOADING PROXY DATA
Will compute a multi-model-prior!
Load d18O  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_d18O_850_1850.nc
computing yearly average
Load evap  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_evap_850_1850.nc
computing yearly average
Load prec  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_prec_850_1850.nc
computing yearly average
Load tsurf  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_tsurf_850_1850.nc
computing yearly average
USING SPELEO PSM
>>>>>>>>>>>>>GETTING MONTHLY d18O Data
>>>>>>>>>>>>>APPLYING KARST FILTER
COMPUTE ANOMALIES IN MODE: BG+HX
Load d18O  from  /home/mchoblet/paleoda/data/model_data/downsampled/ECHAM5_d18O_850_1849.nc
computing yearly average
Load evap  from  /home/mchoblet/paleoda/


100%|██████████| 1/1 [01:26<00:00, 86.73s/it]


Finished multitimescale DA
Save variables
Compute pseudoproxy metrics for  tsurf
Compute pseudoproxy metrics for  d18O
Compute pseudoproxy metrics for  prec
Output written to isoGSM_PP_inf_mme.nc
>>>>>LOADING PROXY DATA
Will compute a multi-model-prior!
Load d18O  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_d18O_850_1850.nc
computing yearly average
Load evap  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_evap_850_1850.nc
computing yearly average
Load prec  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_prec_850_1850.nc
computing yearly average
Load tsurf  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_tsurf_850_1850.nc
computing yearly average
USING SPELEO PSM
>>>>>>>>>>>>>GETTING MONTHLY d18O Data
>>>>>>>>>>>>>APPLYING FRACTIONATION
use mean temperature
>>>>>>>>>>>>>APPLYING KARST FILTER
COMPUTE ANOMALIES IN MODE: BG+HX
Load d18O  from  /home/mchoblet/paleoda/data/model_data/downsampled/ECHAM5_d18O_850_1849.nc
computing y


100%|██████████| 1/1 [01:26<00:00, 86.57s/it]


Finished multitimescale DA
Save variables
Compute pseudoproxy metrics for  tsurf


In [ ]:
def create_dataframe(basepath='/home/mchoblet/paleoda/results/experiments/PPE_speleopsm_nokarst/',
                        models=['iCESM','Echam','GISS','iHadCM3','isoGSM'],
                        priors=['iCESM','mme','Echam','mme','GISS','mme','iHadCM3','mme','isoGSM','mme'],
                        configs=[['no_psm','inf','inf_frac_m','inf_frac_reg']],
                         metric='tsurf_Corr',metric_name='Corr'):
        """
        Returns all global-metrics in a Dataframe form.
        Basepath with / at the end.
        """
        
        #create Dataframe Columns
        models_col=np.repeat(models,4*2,axis=0)
        priors_col=np.repeat(priors,4,axis=0)
        configs_col=np.concatenate(np.repeat(configs,10,axis=0))
        
        #loop over models and compute the metric
        results=[]
        for mod in models:
            #loop over configs
            for c in configs:
                string= basepath+mod + '_PP_' + c +'.nc'
                #load_values:
                data=xr.open_dataset(string)[metric]
                results.append(data.values.reshape(-1))
            
            #now for multimodel ensemble
            #loop over configs
            for c in configs:
                string=basepath+mod + '_PP_'+c+'_mme.nc'
                data=xr.open_dataset(string)[metric]
                results.append(data.values.reshape(-1))
            
        all_data={
            'config': configs_col,
            'values':results,
            'model': models_col,
            'prior':priors_col,}
        
        
        df=pd.DataFrame(all_data)
        return df

def box_plot(df, name='', model='iCESM' ,#['iCESM','Echam','GISS','iHadCM3','isoGSM'], 
             configs=['no psm','inf. weighting','+ frac (mean)','+ frac (regular)'],
             savename=None):
    
    sns.set_palette("colorblind")
    #cut down dataframe to the one model
    
    #dataframe for current model
    df=df[df['model']==model]
    
    fig = plt.figure(figsize=(12,7))
    sns.boxplot(x="config", y="values",
            hue="prior",
            data=df)

    handles, labels = ax.get_legend_handles_labels()
    plt.xticks(np.arange(4),configs)
    plt.xlabel('Configuration',fontsize=20)
    plt.xticks(rotation=45,ha="right",fontsize=16)
    plt.ylabel(metric_name,fontsize=20)
    plt.yticks(fontsize=16)
    l = plt.legend(handles, labels, loc='upper left',frameon=True,fontsize=14)
    
    if savename is not None:
        strg='/home/mchoblet/paleoda/results/plots/'+savename+'.pdf'
        plt.savefig(strg, format='pdf',bbox_inches='tight')


In [ ]:
def icecore_pseudoproxy_nofilt(paths,cfg):
    """
    Paths is the multi-prior variable (where the second one is orography just in case). Hence treat first element of paths, paths[0] as the dict with paths.
    
    """
    cfg_exp=cfg.copy()
    cfg_exp['output_folder']='PPE_icecoreopsm_nofilt'
    cfg_exp['reps']=1
    cfg_exp['obsdata']=['/home/mchoblet/paleoda/data/proxy_dbs/iso2k_db.nc']
    cfg_exp['psm']=['icecore']
    cfg_exp['ppe']['SNR']=[0.5]
    cfg_exp['ppe']['psm']['icecore']={
    'weighting':'prec', #None(pure d18O) / prec (d180 prec weighted)
    'height': False, #height correction: True requires orography of model and 'elev' coordinate for proxy_db. Requires that proxy_db has a coordinate 'lapse rate' (e.g. i attributed -0.15 to non polar porxies and -0.28 to polar ones (per 100m)
    'filter': False, #If True the PSM-function from PRYSM which takes into account diffusion is applied. 
}
    
        
    #loop over proxy origin
    for i1,p1 in tqdm.tqdm(paths[0].items()):
        cfg_exp['multi_model_prior']=None
        cfg_exp['ppe']['other_model']['vp']=p1
  
        #1. no-psm at all
        cfg_exp['icecore']={
    'weighting':None, #None(pure d18O) / prec (d180 prec weighted)
    'height': False, #height correction: True requires orography of model and 'elev' coordinate for proxy_db. Requires that proxy_db has a coordinate 'lapse rate' (e.g. i attributed -0.15 to non polar porxies and -0.28 to polar ones (per 100m)
    'filter': False, #If True the PSM-function from PRYSM which takes into account diffusion is applied. 
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'no_psm'
        wrapper.paleoda_wrapper(cfg_exp)
        
        #2. infiltration weighting
        cfg_exp['icecore']={
    'weighting':'prec', #None(pure d18O) / prec (d180 prec weighted)
    'height': False, #height correction: True requires orography of model and 'elev' coordinate for proxy_db. Requires that proxy_db has a coordinate 'lapse rate' (e.g. i attributed -0.15 to non polar porxies and -0.28 to polar ones (per 100m)
    'filter': False, #If True the PSM-function from PRYSM which takes into account diffusion is applied. 
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'prec'
        wrapper.paleoda_wrapper(cfg_exp)


        #the same using a multi- model prior (leave original model out)
        copied_paths=copy.deepcopy(multi_prior_ds[0])
        #delete current proxy origin
        del copied_paths[i1]
        #put back into list form
        copied_paths=[copied_paths]
        cfg_exp['multi_model_prior']=copied_paths

                #1. no-psm at all
   
               #1. no-psm at all
        cfg_exp['icecore']={
    'weighting':None, #None(pure d18O) / prec (d180 prec weighted)
    'height': False, #height correction: True requires orography of model and 'elev' coordinate for proxy_db. Requires that proxy_db has a coordinate 'lapse rate' (e.g. i attributed -0.15 to non polar porxies and -0.28 to polar ones (per 100m)
    'filter': False, #If True the PSM-function from PRYSM which takes into account diffusion is applied. 
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'no_psm_mme'
        wrapper.paleoda_wrapper(cfg_exp)
        
        #2. infiltration weighting
        cfg_exp['icecore']={
    'weighting':'prec', #None(pure d18O) / prec (d180 prec weighted)
    'height': False, #height correction: True requires orography of model and 'elev' coordinate for proxy_db. Requires that proxy_db has a coordinate 'lapse rate' (e.g. i attributed -0.15 to non polar porxies and -0.28 to polar ones (per 100m)
    'filter': False, #If True the PSM-function from PRYSM which takes into account diffusion is applied. 
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'prec_mme'
        wrapper.paleoda_wrapper(cfg_exp)

        
        

In [ ]:
icecore_pseudoproxy_nofilt(multi_prior_ds,cfg)

In [32]:
def icecore_pseudoproxy_filt(paths,cfg):
    """
    Paths is the multi-prior variable (where the second one is orography just in case). Hence treat first element of paths, paths[0] as the dict with paths.
    
    """
    cfg_exp=cfg.copy()
    cfg_exp['output_folder']='PPE_icecoreopsm_filt'
    cfg_exp['reps']=1
    cfg_exp['obsdata']=['/home/mchoblet/paleoda/data/proxy_dbs/iso2k_db.nc']
    cfg_exp['psm']=['icecore']

    cfg_exp['ppe']['psm']['icecore']={
    'weighting':'prec', #None(pure d18O) / prec (d180 prec weighted)
    'height': False, #height correction: True requires orography of model and 'elev' coordinate for proxy_db. Requires that proxy_db has a coordinate 'lapse rate' (e.g. i attributed -0.15 to non polar porxies and -0.28 to polar ones (per 100m)
    'filter': True, #If True the PSM-function from PRYSM which takes into account diffusion is applied. 
}
    
    #here choose extremely high snr
    cfg_exp['ppe']['SNR']=[10]
    
        
    #loop over proxy origin
    for i1,p1 in tqdm.tqdm(paths[0].items()):
        cfg_exp['multi_model_prior']=None
        cfg_exp['ppe']['other_model']['vp']=p1
        
        #1. no-psm at all
        cfg_exp['icecore']={
    'weighting':None, #None(pure d18O) / prec (d180 prec weighted)
    'height': False, #height correction: True requires orography of model and 'elev' coordinate for proxy_db. Requires that proxy_db has a coordinate 'lapse rate' (e.g. i attributed -0.15 to non polar porxies and -0.28 to polar ones (per 100m)
    'filter': False, #If True the PSM-function from PRYSM which takes into account diffusion is applied. 
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'no_psm'
        wrapper.paleoda_wrapper(cfg_exp)
        
        #2. infiltration weighting
        cfg_exp['icecore']={
    'weighting':'prec', #None(pure d18O) / prec (d180 prec weighted)
    'height': False, #height correction: True requires orography of model and 'elev' coordinate for proxy_db. Requires that proxy_db has a coordinate 'lapse rate' (e.g. i attributed -0.15 to non polar porxies and -0.28 to polar ones (per 100m)
    'filter': False, #If True the PSM-function from PRYSM which takes into account diffusion is applied. 
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'prec'
        wrapper.paleoda_wrapper(cfg_exp)
        

        #3. with diffusion and full-psm
        cfg_exp['icecore']={
    'weighting':'prec', #None(pure d18O) / prec (d180 prec weighted)
    'height': False, #height correction: True requires orography of model and 'elev' coordinate for proxy_db. Requires that proxy_db has a coordinate 'lapse rate' (e.g. i attributed -0.15 to non polar porxies and -0.28 to polar ones (per 100m)
    'filter': True, #If True the PSM-function from PRYSM which takes into account diffusion is applied. 
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'filt'
        wrapper.paleoda_wrapper(cfg_exp)
        
        #the same using a multi- model prior (leave original model out)
        copied_paths=copy.deepcopy(multi_prior_ds[0])
        #delete current proxy origin
        del copied_paths[i1]
        #put back into list form
        copied_paths=[copied_paths]
        cfg_exp['multi_model_prior']=copied_paths


        #1. no-psm at all
        cfg_exp['icecore']={
    'weighting':None, #None(pure d18O) / prec (d180 prec weighted)
    'height': False, #height correction: True requires orography of model and 'elev' coordinate for proxy_db. Requires that proxy_db has a coordinate 'lapse rate' (e.g. i attributed -0.15 to non polar porxies and -0.28 to polar ones (per 100m)
    'filter': False, #If True the PSM-function from PRYSM which takes into account diffusion is applied. 
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'no_psm_mme'
        wrapper.paleoda_wrapper(cfg_exp)
        
        #2. infiltration weighting
        cfg_exp['icecore']={
    'weighting':'prec', #None(pure d18O) / prec (d180 prec weighted)
    'height': False, #height correction: True requires orography of model and 'elev' coordinate for proxy_db. Requires that proxy_db has a coordinate 'lapse rate' (e.g. i attributed -0.15 to non polar porxies and -0.28 to polar ones (per 100m)
    'filter': False, #If True the PSM-function from PRYSM which takes into account diffusion is applied. 
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'prec_mme'
        wrapper.paleoda_wrapper(cfg_exp)
        
        #3. with diffusion and full-psm
        cfg_exp['icecore']={
    'weighting':'prec', #None(pure d18O) / prec (d180 prec weighted)
    'height': False, #height correction: True requires orography of model and 'elev' coordinate for proxy_db. Requires that proxy_db has a coordinate 'lapse rate' (e.g. i attributed -0.15 to non polar porxies and -0.28 to polar ones (per 100m)
    'filter': True, #If True the PSM-function from PRYSM which takes into account diffusion is applied. 
}
        cfg_exp['output_file']=str(i1)+'_PP_'+'filt_mme'
        wrapper.paleoda_wrapper(cfg_exp)


In [ ]:
icecore_pseudoproxy_filt(multi_prior_ds,cfg)

  0%|          | 0/5 [00:00<?, ?it/s]

>>>>>LOADING PROXY DATA
Load d18O  from  /home/mchoblet/paleoda/data/model_data/ECHAM5_d18O_850_1849.nc
computing yearly average
Checking prior for nans
Load prec  from  /home/mchoblet/paleoda/data/model_data/ECHAM5_prec_850_1849.nc
computing yearly average
Checking prior for nans
Load tsurf  from  /home/mchoblet/paleoda/data/model_data/ECHAM5_tsurf_850_1849.nc
computing yearly average
Checking prior for nans
Load evap  from  /home/mchoblet/paleoda/data/model_data/ECHAM5_evap_850_1849.nc
computing yearly average
Checking prior for nans
USING ICECORE PSM
COMPUTE ANOMALIES IN MODE: BG+HX
GENERATING PSEUDOPROXIES FROM MODEL OTHER THAN PRIOR
Changed Pseudoproxy-PSM-Config for  speleo
Changed Pseudoproxy-PSM-Config for  icecore
Load d18O  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_d18O_850_1850.nc
computing yearly average
Checking prior for nans
Load evap  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_evap_850_1850.nc
computing yearly average
Checking prio


100%|██████████| 117/117 [05:03<00:00,  2.59s/it]


COMPUTE ANOMALIES IN MODE: BG+HX
Start Multitimescale DA loop.



100%|██████████| 1/1 [00:13<00:00, 13.39s/it]


...


100%|██████████| 117/117 [06:42<00:00,  3.44s/it]


COMPUTE ANOMALIES IN MODE: BG+HX
Start Multitimescale DA loop.



100%|██████████| 1/1 [00:16<00:00, 16.33s/it]


Finished multitimescale DA
Save variables
Compute pseudoproxy metrics for  tsurf
Compute pseudoproxy metrics for  d18O
Compute pseudoproxy metrics for  prec
Output written to GISS_PP_filt.nc
>>>>>LOADING PROXY DATA
Will compute a multi-model-prior!
Load d18O  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_d18O_850_1850.nc
computing yearly average
Load evap  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_evap_850_1850.nc
computing yearly average
Load prec  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_prec_850_1850.nc
computing yearly average
Load tsurf  from  /home/mchoblet/paleoda/data/model_data/downsampled/CESM_tsurf_850_1850.nc
computing yearly average
USING ICECORE PSM
COMPUTE ANOMALIES IN MODE: BG+HX
Load d18O  from  /home/mchoblet/paleoda/data/model_data/downsampled/CCSM_d18O_851_1850.nc
computing yearly average
Load evap  from  /home/mchoblet/paleoda/data/model_data/downsampled/CCSM_evap_851_1850.nc
computing yearly average
Load pre


 21%|██▏       | 25/117 [01:13<05:29,  3.58s/it]